In [1]:
from datetime import datetime
import shutil
import torch
import numpy as np
import rasterio
import os
from rasterio.windows import Window
from time import sleep

# DONE: Made test loader georeference LST
# DONE: Make test loader georeference all the rest to check ground truth against original as tile
# DONE: Test making each tile
# DONE: Test 0.0 overlap
# DONE: Organize to inference script and test the data loading more
# DONE: Make model prediction also pass geo reference and test against original as tile
# DONE: Add Heat index 1-25 into preprocess
# DONE: Make Heat Index show up in test quality
# DONE: Make it show in inference
# DONE: Train a preliminary model with 20 epochs, batch 1
# DONE: Test the model in inference
# DONE: Create a nowcast, 3 month cast and 6 month cast according to some parameter
# DONE: Get batching working -> max out the VRAM
# UNECESSARY: In inference, combine the tiles by city. Make it possible to do partial. Don't combine not in same city
# DONE: Make sure 512 and batching does have proper loss
# DONE: Profile the pipeline
# DONE: Get stats of all
# DONE: Preprocess all to float 32 with same nodata -9999 AND test
# DONE: Implement normalization
# DONE: Test data quality of data loader
# DONE: Fix denormalize
# DONE: Fix Tiled
# DONE: Gut and remove LandsatDataModule to TiledLandsatDataModule
# DONE: Preprocess all
# DONE: Add true normalization
# DONE: Create saving checkpoints by folder, test one 1 epoch
# DONE: Use Wandb with alert and Slack, setup phone
# DONE: Train and test with augmentation and batch sizing
# DONE: Test all 4 models for test loader accuracy, no artifact saved. Instead include a test loader test whenever val F is at its best. Make each on connect in the graph
# DONE: Test proper loss with normalization
# DONE: Fix labelling, metrics and inferencing. Test validate and train
# DONE: Test and correct inference
# DONE: Validate quality of 3 month and 6 month ahead data loaders
# DONE: Change all to 128
# DONE: Double check quality of 128. Do all three loaders then all three loaders again with augmentation. Use like 45 tiles.
# DONE: Do a full debug run to check underfitting
# DONE: Preprocess new NDBI
# DONE: Quality check NDBI
# DONE: Mixed precision
# DONE: NDBI upload
# DONE: Check 3 month 6 month for 128 dimensions then full dimension consistencies, then check tile accruacies across 100 tiles for 3 montha dn 6 month
# DONE: Compare cityscapes b5 with regular b5 segformer
# DONE: Try OneFormer
# DONE: Do 3 month ahead and 6 month ahead Segformer debug runs.
# TODO: Make inference do a while scene and merge to it
# TODO: Inference 3 month model and compare to ground truh in ArcGIS Pro
# DONE: Add deeplab
# TODO: Upload to huggingface
# TODO: Do complete experiment
# DONE: Compare to the good originals
# DONE: Check with Isaac's recommendations
# DONE: Double check normalization, tile size
# DONE: Debug underfitting in debug
# DONE: Try one FULL data run on the 4 GPUs
# DONE: Debug full run
# DONE: Fix mixed precision
# DONE: Test debug run on mixed precision
# DONE: Test full run on mixed precision
# DONE: Tune batch size
# DONE: Train one good model on 3 month prediction Batch 32, Augment True, debug true
# TODO: Compare 3 month accuracy to 0 month
# DONE: Migrate to Jetstream
# DONE: Create a way to look 0, 3, 6 months ahead
# TODO: Test by zero-shot, city only, temporal only, both, debug true, Use the table
# TODO: Debug False, Test by zero-shot, city only, temporal only, both, Use the table
# TODO: Report to Isaac and show table
#*Merging in inference
#*Maybe use open street map
#*Maybe use NDBI


In [2]:
from utils.data.TiledLandsatDataModule import TiledGeotiffDataset
import glob

def combine_scene_tifs(scene_id_folder):
    """Combine LST and HeatIndex TIF files using rasterio.merge."""
    import rasterio
    from rasterio.merge import merge
    import glob
    import os
    
    os.makedirs(scene_id_folder, exist_ok=True)
    
    for outTif in ['LST.tif', 'HeatIndex.tif']:
        tif_files = glob.glob(os.path.join(scene_id_folder, f'predicted_*_{outTif}'))
        print(f"Found {len(tif_files)} files for {outTif}")
        print(tif_files)
        
        if not tif_files:
            print(f"No {outTif} files found in {scene_id_folder}")
            continue
            
        # Open all files
        src_files = [rasterio.open(f) for f in tif_files]
        
        # Merge them
        mosaic, out_transform = merge(src_files, nodata=-9999)
        
        # Get metadata from first file
        profile = src_files[0].profile.copy()
        profile.update({
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_transform,
            "nodata": -9999
        })
        
        # Save merged result
        output_file = os.path.join(os.path.dirname(scene_id_folder), 
                                  f"{os.path.basename(scene_id_folder)}_combined_{outTif}")
        with rasterio.open(output_file, 'w', **profile) as dst:
            dst.write(mosaic)
            
        print(f"Combined {outTif} saved as: {output_file}")
        
        # Close the open files
        for src in src_files:
            src.close()


def inference(model, test_loader, tiles_count: int, device='cuda', denormalize: bool=False):
    model = model.to(device)
    model.eval()    
    batch = 0
    it = iter(test_loader)
    scenes = set()
    with torch.no_grad():
        for _ in range(tiles_count):
            sleep(1)
            # Get one sample
            sample = next(it)
            for l, outTif in enumerate(['LST.tif', 'HeatIndex.tif']):
                inputs = sample['input'].to(device)
                targets = sample['target'].to(device)
                mask = sample['mask'].to(device)
                ground_truth_file = sample['file_dict'][outTif][0]
                box = sample['box']
                box = [int(tensor.item()) for tensor in box]

                # Get model prediction
                outputs = model(inputs)
                if denormalize:
                    print(outputs.shape)
                    outputs = TiledGeotiffDataset.denormalize(outputs)[batch][l]
                    targets = TiledGeotiffDataset.denormalize(targets)[batch][l]

                # Move to CPU and convert to numpy
                mask_np = mask.cpu().numpy().squeeze()
                targets_np = targets.cpu().numpy().squeeze()
                predicted_np = outputs.cpu().numpy().squeeze()
                
                # Apply mask
                predicted_np[~mask_np] = -9999
                targets_np[~mask_np] = -9999

                output_dir = "./Data/prediction"
                os.makedirs(output_dir, exist_ok=True)
                xmin, ymin, xmax, ymax = box
                window = Window(col_off=xmin, row_off=ymin, width=xmax-xmin, height=ymax-ymin)
                
                # Get corresponding LST file path and save outputs
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                with rasterio.open(ground_truth_file) as src:
                    profile = src.profile.copy()
                    # Update the transform based on the window
                    window_transform = rasterio.windows.transform(window, src.transform)
                    
                    # Update profile with new dimensions and transform
                    profile.update(
                        width=xmax-xmin,
                        height=ymax-ymin,
                        transform=window_transform,
                        count=1,
                        nodata=-9999
                    )                                                    
                    
                    # Save prediction
                    scene_id = f"{ground_truth_file.split('/')[-3]}_{ground_truth_file.split('/')[-2]}"  # city_date
                    scenes.add(os.path.join(output_dir, scene_id))
                    os.makedirs(os.path.join(output_dir, scene_id), exist_ok=True)
                    pred_filename = os.path.join(output_dir, scene_id, f'predicted_{timestamp}_{outTif}')
                    with rasterio.open(pred_filename, "w", **profile) as dst:
                        dst.write(predicted_np.astype(np.float32), 1)
                    
                    # Save ground truth
                    os.makedirs(os.path.join(output_dir, f'ground_truth_{scene_id}'), exist_ok=True)
                    truth_filename = os.path.join(output_dir, f'ground_truth_{scene_id}', f'ground_truth_{timestamp}_{outTif}')
                    with rasterio.open(truth_filename, "w", **profile) as dst:
                        dst.write(targets_np.astype(np.float32), 1)
                    
                    # Copy original out file
                    orig_filename = os.path.join(output_dir, f'original_{scene_id}_{outTif}')                    
                    if not os.path.exists(orig_filename):
                        print(f"Original LST: {os.path.basename(orig_filename)}")
                        shutil.copy2(ground_truth_file, orig_filename)
                    
                    # Calculate metrics for valid pixels
                    valid_mask = predicted_np != -9999
                    if valid_mask.any():
                        mae = np.mean(np.abs(predicted_np[valid_mask] - targets_np[valid_mask]))
                        rmse = np.sqrt(np.mean((predicted_np[valid_mask] - targets_np[valid_mask])**2))
                        metrics = {'mae': mae, 'rmse': rmse}
                        print(f"Predictions: {os.path.basename(pred_filename)}")
                        # print(f"Ground Truth: {os.path.basename(truth_filename)}")
                        if 'Heat' in pred_filename:
                            print(f"Mean Absolute Error: {mae:.2f} points.")
                            print(f"Root Mean Square Error: {rmse:.2f} points.")
                        else:
                            print(f"Mean Absolute Error: {mae:.2f}°F")
                            print(f"Root Mean Square Error: {rmse:.2f}°F")
                print(f"\nSaved files in {output_dir}/:")
    print(f'There are {len(scenes)} scenes')
    for scene in scenes:
        combine_scene_tifs(scene)
    
    # return metrics

def test_data_quality(test_loader, tiles_count: int, denormalize: bool = False):
    batch = 0
    it = iter(test_loader)
    for _ in range(tiles_count):
        sleep(1)
        # Get one sample
        sample = next(it)
        printedOriginals = set()
        if denormalize:
            sample = TiledGeotiffDataset.denormalize(sample)
        for i, tif in enumerate(['Albedo.tif', 'DEM.tif', 'Land_Cover.tif', 'NDVI.tif', 'NDWI.tif', 'NDBI.tif', 'LST.tif', 'HeatIndex.tif']):
            with torch.no_grad():
                if i <= 5:
                    targets = sample['input'][batch][i]
                else:
                    targets = sample['target'][batch][i-6]
                mask = sample['mask']
                target_file_origin = sample['file_dict'][tif][0]
                scene_id = str(target_file_origin.split('/')[-3]) + '_' + str(target_file_origin.split('/')[-2]) + '_' + str(tif)
                box = sample['box']
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                box = [int(tensor.item()) for tensor in box]
                # print(box)

                # Move to CPU and convert to numpy
                mask_np = mask.cpu().numpy().squeeze()
                targets_np = targets.cpu().numpy().squeeze()

                noData = -9999
                targets_np[~mask_np] = noData
                # Save ground truth
                output_dir = "./Data/truth"
                os.makedirs(output_dir, exist_ok=True)
                xmin, ymin, xmax, ymax = box
                window = Window(col_off=xmin, row_off=ymin, width=xmax-xmin, height=ymax-ymin)
                
                print(target_file_origin)
                with rasterio.open(target_file_origin) as src:
                    # Get the profile from the source
                    test_profile = src.profile.copy()
                    
                    # Update the transform based on the window
                    window_transform = rasterio.windows.transform(window, src.transform)
                    
                    # Update profile with new dimensions and transform
                    test_profile.update(
                        width=xmax-xmin,
                        height=ymax-ymin,
                        transform=window_transform,
                        count=1,
                        nodata=noData
                    )
                    
                    truth_filename = os.path.join(output_dir, f'ground_truth_{timestamp}_{tif}')
                    with rasterio.open(truth_filename, "w", **test_profile) as dst:
                        dst.write(targets_np.astype(np.float32), 1)

                orig_filename = os.path.join(output_dir, f'original_File_{scene_id}')
                if os.path.exists(orig_filename):
                    print(f"Original file already exists: {orig_filename}")
                    continue
                with rasterio.open(target_file_origin) as src:
                    profile = src.profile.copy()
                    profile.update(dtype='float32', count=1, nodata=np.nan)
                    print(f"\nSaved files in {output_dir}/:")
                    print(f"Ground Truth: {os.path.basename(truth_filename)}")
                    print(f"Original LST: {os.path.basename(orig_filename)}")
                    # Copy original output files
                    print(f'Copying {target_file_origin} to {orig_filename}')
                    if not os.path.exists(orig_filename):
                        shutil.copy2(target_file_origin, orig_filename)

def test_sizes(test_loader, num_samples=None):
    """
    Test if all X, y from the dataloader have the same image sizes.
    
    Args:
        test_loader: DataLoader to test
        num_samples: Number of samples to check (default: all)
    
    Returns:
        bool: True if all consistent, False otherwise
    """
    print("Testing image sizes in dataloader...")
    
    # Get the first batch to establish expected shapes
    it = iter(test_loader)
    first_sample = next(it)
    
    # Extract shapes
    input_shape = first_sample['input'].shape
    target_shape = first_sample['target'].shape
    mask_shape = first_sample['mask'].shape
    
    print(f"Expected input shape: {input_shape}")
    print(f"Expected target shape: {target_shape}")
    print(f"Expected mask shape: {mask_shape}")
    
    # Check if the shapes are as expected (6 input channels, 2 target channels)
    if input_shape[1] != 6:
        print(f"WARNING: Expected 6 input channels, got {input_shape[1]}")
    if target_shape[1] != 2:
        print(f"WARNING: Expected 2 target channels, got {target_shape[1]}")
    
    # Reset iterator
    it = iter(test_loader)
    all_consistent = True
    count = 0
    
    # Define the expected input files
    expected_files = ['Albedo.tif', 'DEM.tif', 'Land_Cover.tif', 'NDVI.tif', 'NDWI.tif', 'NDBI.tif', 'LST.tif', 'HeatIndex.tif']
    
    # Iterate through samples
    total_samples = len(test_loader) if num_samples is None else min(num_samples, len(test_loader))
    
    for i in range(total_samples):
        try:
            sample = next(it)
            
            # Check shapes
            current_input_shape = sample['input'].shape
            current_target_shape = sample['target'].shape
            current_mask_shape = sample['mask'].shape
            
            # Verify file dictionary contains all expected files
            file_dict = sample['file_dict']
            missing_files = [f for f in expected_files if f not in file_dict]
            
            if current_input_shape != input_shape:
                print(f"Sample {i}: Input shape mismatch! Expected {input_shape}, got {current_input_shape}")
                all_consistent = False
            
            if current_target_shape != target_shape:
                print(f"Sample {i}: Target shape mismatch! Expected {target_shape}, got {current_target_shape}")
                all_consistent = False
            
            if current_mask_shape != mask_shape:
                print(f"Sample {i}: Mask shape mismatch! Expected {mask_shape}, got {current_mask_shape}")
                all_consistent = False
            
            if missing_files:
                print(f"Sample {i}: Missing files: {missing_files}")
                all_consistent = False
            
            # Check each file in the file_dict has consistent number of entries
            for file_type, file_list in file_dict.items():
                if len(file_list) != 1:  # Assuming each should have exactly 1 file
                    print(f"Sample {i}: File type {file_type} has {len(file_list)} entries instead of 1")
                    all_consistent = False
            
            # Print progress
            count += 1
            if count % 10 == 0:
                print(f"Checked {count}/{total_samples} samples...")
                
        except StopIteration:
            break
    
    # Final summary
    if all_consistent:
        print(f"✅ All {count} samples have consistent shapes!")
        print(f"Input shape: {input_shape}")
        print(f"Target shape: {target_shape}")
        print(f"Mask shape: {mask_shape}")
    else:
        print(f"❌ Found inconsistencies in the {count} samples checked.")
    
    return all_consistent

import os
import rasterio
from tqdm import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
def get_file_paths(folder_path: str) -> list[str]:
        file_paths = []
        for root, _, files in os.walk(folder_path):
            for file in files:
                full_path = os.path.abspath(os.path.join(root, file))
                file_paths.append(full_path)
        return file_paths   
def test_original_file_dimensions(data_dir, months_ahead=3, include_years=None):
    """
    Test if all original TIF files in X and y folders have consistent dimensions.
    
    Args:
        data_dir: Base data directory
        months_ahead: Months ahead for prediction (0, 3, 6)
        include_years: List of years to include (e.g., ["2018", "2019"])
    """
    if include_years is None:
        include_years = ["2018", "2019", "2020"]
    
    print(f"Testing original TIF files in preprocess_{months_ahead}monthsahead...")
    
    # Expected input and output files
    input_files = ['Albedo.tif', 'DEM.tif', 'Land_Cover.tif', 'NDVI.tif', 'NDWI.tif', 'NDBI.tif']
    output_files = ['LST.tif', 'HeatIndex.tif']
    
    # Store file dimensions for each scene
    scene_dimensions = {}
    inconsistent_scenes = []
    
    # Find all Albedo files as hooks
    albedo_files = []
    x_dir = os.path.join(data_dir, f'preprocess_{months_ahead}monthsahead', 'X', 'less5CloudCover')
    for file_path in tqdm(get_file_paths(x_dir), desc='Gathering scenes...'):
        date = file_path.split('/')[-2]
        for year in include_years:
            if year in date:
                if 'Albedo' in file_path:
                    albedo_files.append(file_path)
    
    print(f"Found {len(albedo_files)} Albedo files to check")
    for albedo_path in tqdm(albedo_files, desc='Checking file dimensions'):
        # Get scene info from albedo path
        scene_dir = os.path.dirname(albedo_path)
        scene_id = f"{scene_dir.split('/')[-3]}_{scene_dir.split('/')[-2]}"  # city_date
        
        # Initialize dimension data for this scene
        scene_dimensions[scene_id] = {'input': {}, 'output': {}}
        has_consistency = True
        
        # Check input files
        for input_file in input_files:
            input_path = os.path.join(scene_dir, input_file)
            if os.path.exists(input_path):
                try:
                    with rasterio.open(input_path) as src:
                        width, height = src.width, src.height
                        scene_dimensions[scene_id]['input'][input_file] = (width, height)
                except Exception as e:
                    print(f"Error reading {input_path}: {e}")
                    has_consistency = False
            else:
                print(f"Missing input file: {input_path}")
                has_consistency = False
        
        # Check corresponding output files
        date = scene_dir.split('/')[-1]
        date_object = datetime.strptime(date, "%Y-%m")
        date_object = date_object + relativedelta(months=months_ahead)
        dateAhead = date_object.strftime("%Y-%m")
        y_dir = albedo_path.replace('/X/', '/y/').replace(date, dateAhead).replace('Albedo.tif', '')
        for output_file in output_files:
            output_path = os.path.join(y_dir, output_file)
            if os.path.exists(output_path):
                try:
                    with rasterio.open(output_path) as src:
                        width, height = src.width, src.height
                        scene_dimensions[scene_id]['output'][output_file] = (width, height)
                except Exception as e:
                    print(f"Error reading {output_path}: {e}")
                    has_consistency = False
            else:
                print(f"Missing output file: {output_path}")
                has_consistency = False

    for id in tqdm(scene_dimensions.keys(), desc='Checking for inconsistencies'):
        input = set(scene_dimensions[id]['input'].values())
        output = set(scene_dimensions[id]['output'].values())
        if len(input | output) > 1:
            inconsistent_scenes.append(id)
            print(f"Error: {id} has different sizes")
            has_consistency = False
    print("The consistency is " + str(has_consistency))


    

In [3]:
# import wandb
# import os

# os.environ["WANDB_NOTEBOOK_NAME"] = "TrainUNet-Basic.ipynb"
# os.environ["WANDB_DIR"] = "./wandb"
# os.environ["WANDB_CACHE_DIR"] = "/work/ubh496/.cache/wandb"
# os.environ["WANDB_CONFIG_DIR"] = "/work/ubh496/.config/wandb"
# os.environ["WANDB_DATA_DIR"] = "/work/ubh496/.cache/wandb-data"
# os.environ["WANDB_ARTIFACT_DIR"] = "./artifacts"

# run = wandb.init(dir="/work/ubh496/heat-island-test/wandb/downloaded_models")
# artifact = run.use_artifact('jesus-guerrero-ml/heat-island/model-sclb910d:v8', type='model')
# artifact_dir = artifact.download()

In [4]:
from utils.data.TiledLandsatDataModule import TiledLandsatDataModule
from utils.model import LSTNowcaster

config = {
    "debug": True,
    "augment": False,
    "by_city": False,
    "tile_size": 128,
    "tile_overlap": 0.0,
    "months_ahead": 3,
    "learning_rate": 1e-4,
    "model": "unet",
    "backbone": "resnet50",
    "dataset": "pure_landsat",
    "epochs": 25,
    "batch_size": 1,
    "pretrained_weights": True,
    "deterministic": True,
    "in_channels": 6
}

best_model = LSTNowcaster()

best_model = LSTNowcaster.load_from_checkpoint(
    "/home/ubuntu/heat-island-test/wandb/heat-island/checkpoints/up47iayb_April15/up47iayb_April15_epoch=059_val_rmse_F=17.0594.ckpt"
)

data_module = TiledLandsatDataModule(
    data_dir="./Data",
    monthsAhead=config["months_ahead"],
    augment=config["augment"],
    shuffleTrain=False,
    batch_size=1,
    num_workers=5,        
    tile_size=config["tile_size"],
    includeYears=["2018", "2019", "2020"]
)
# Setup the data module to prepare datasets
data_module.setup()
print(f'Test is length {len(data_module.test_dataloader())}')
print(f'Validate is length {len(data_module.val_dataloader())}')
print(f'Train is length {len(data_module.train_dataloader())}')


Preparing scene by scene...: 100%|██████████| 1156/1156 [00:00<00:00, 10915.04it/s]


Dataset splits - Train: 924, Val: 116, Test: 116
Test is length 5435
Validate is length 4367
Train is length 43246


In [5]:
# scene_dimensions = test_original_file_dimensions(
#     data_dir="./Data",
#     months_ahead=6,
#     include_years=["2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
# )

# test_data_quality(
#     test_loader=data_module.test_dataloader(), tiles_count=100, denormalize=True
# )

inference(
    model=best_model, test_loader=data_module.test_dataloader(), tiles_count=45, denormalize=True
)

# Test all dataloaders for consistent image sizes
# print("Testing train dataloader...")
# train_consistent = test_sizes(data_module.train_dataloader())

# print("\nTesting validation dataloader...")
# val_consistent = test_sizes(data_module.val_dataloader())

# print("\nTesting test dataloader...")
# test_consistent = test_sizes(data_module.test_dataloader())

# # Summary of all tests
# print("\n=== SUMMARY ===")
# print(f"Train dataloader consistent: {'Yes ✅' if train_consistent else 'No ❌'}")
# print(f"Validation dataloader consistent: {'Yes ✅' if val_consistent else 'No ❌'}")
# print(f"Test dataloader consistent: {'Yes ✅' if test_consistent else 'No ❌'}")

# if train_consistent and val_consistent and test_consistent:
#     print("\nAll dataloaders have consistent image sizes! ✅")
# else:
#     print("\nWARNING: Inconsistencies detected in one or more dataloaders! ❌")
#     print("Fix the inconsistencies before proceeding with training.")

/home/ubuntu/heat-island-test/utils/model.py:104: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


torch.Size([1, 2, 128, 128])
Original LST: original_Sierra_Vista_AZ_2018-12_LST.tif
Predictions: predicted_20250416_012709_LST.tif
Mean Absolute Error: 25.64°F
Root Mean Square Error: 25.70°F

Saved files in ./Data/prediction/:
torch.Size([1, 2, 128, 128])
Original LST: original_Sierra_Vista_AZ_2018-12_HeatIndex.tif
Predictions: predicted_20250416_012709_HeatIndex.tif
Mean Absolute Error: 1.14 points.
Root Mean Square Error: 1.46 points.

Saved files in ./Data/prediction/:
torch.Size([1, 2, 128, 128])
Predictions: predicted_20250416_012711_LST.tif
Mean Absolute Error: 26.36°F
Root Mean Square Error: 26.46°F

Saved files in ./Data/prediction/:
torch.Size([1, 2, 128, 128])
Predictions: predicted_20250416_012711_HeatIndex.tif
Mean Absolute Error: 1.23 points.
Root Mean Square Error: 1.66 points.

Saved files in ./Data/prediction/:
torch.Size([1, 2, 128, 128])
Predictions: predicted_20250416_012712_LST.tif
Mean Absolute Error: 25.31°F
Root Mean Square Error: 25.39°F

Saved files in ./Data/